In [1]:
import optuna
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

print("All imports succeeded!")

d:\c\c\ML_P\test_1\env_MLP_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports succeeded!


In [2]:
# ---------------------------------------------------------------------------------
# Step 1: Load the Pima Indians Diabetes dataset from a URL
# ---------------------------------------------------------------------------------

# URL to the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Column names based on dataset documentation
columns = [
    "Pregnancies", "Glucose", "BloodPressure", "SkinThickness", 
    "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"
]

# Read CSV data from the URL and assign column names
data = pd.read_csv(url, header=None, names=columns)

# Separate the features (X) and the target variable (y)
X = data.drop("Outcome", axis=1)
y = data["Outcome"]


In [3]:

# ---------------------------------------------------------------------------------
# Step 2: Define the objective function for Optuna optimization
# ---------------------------------------------------------------------------------

def objective(trial):
    """
    Objective function for Optuna to maximize accuracy.
    This function dynamically selects a classifier and its corresponding
    hyperparameters based on the trial suggestion.
    """
    # Dynamically choose the classifier using a categorical suggestion
    classifier_name = trial.suggest_categorical("classifier", ["xgb",
                                                                "svm",
                                                                #   "nn"
                                                                  ])
    
    if classifier_name == "xgb":
        # ---------------------------------------------------------------------
        # XGBoost Classifier: Define model-specific hyperparameter search space
        # ---------------------------------------------------------------------
        n_estimators = trial.suggest_int("xgb_n_estimators", 50, 300)
        max_depth = trial.suggest_int("xgb_max_depth", 3, 10)
        learning_rate = trial.suggest_float("xgb_learning_rate", 1e-3, 1.0, log=True)
        subsample = trial.suggest_float("xgb_subsample", 0.5, 1.0)
        classifier = XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            # max_iter=1000,
            # tol=1e-4,
            # use_label_encoder=False,  # Avoid warnings regarding label encoding
            eval_metric="logloss"     # Evaluation metric for binary classification
        )
    elif classifier_name == "svm":
        # ---------------------------------------------------------------------
        # SVM Classifier: Define model-specific hyperparameter search space
        # ---------------------------------------------------------------------
        c = trial.suggest_float("svm_C", 1e-3, 1e3, log=True)
        kernel = trial.suggest_categorical("svm_kernel", [
            # "linear",
              "rbf"
              ])
        if kernel == "rbf":
            # For RBF kernel, include gamma in the search space
            gamma = trial.suggest_float("svm_gamma", 1e-4, 1e-1, log=True)
            classifier = SVC(C=c, kernel=kernel, gamma=gamma, probability=True)
        else:
            classifier = SVC(C=c, kernel=kernel, probability=True)
    # else:
    #     # ---------------------------------------------------------------------
    #     # Neural Network (MLPClassifier): Define model-specific hyperparameter search space
    #     # ---------------------------------------------------------------------
    #     hidden_layer_sizes = trial.suggest_categorical("nn_hidden_layer_sizes", [(50,), (100,), (50, 50)])
    #     activation = trial.suggest_categorical("nn_activation", ["relu", "tanh"])
    #     alpha = trial.suggest_float("nn_alpha", 1e-5, 1e-1, log=True)
    #     learning_rate_init = trial.suggest_float("nn_learning_rate_init", 1e-4, 1e-1, log=True)
    #     classifier = MLPClassifier(
    #         hidden_layer_sizes=hidden_layer_sizes,
    #         activation=activation,
    #         alpha=alpha,
    #         learning_rate_init=learning_rate_init,
    #         max_iter=500  # Set maximum iterations to ensure convergence
    #     )
    
    # ---------------------------------------------------------------------------------
    # Step 3: Evaluate the classifier using 5-fold cross-validation
    # ---------------------------------------------------------------------------------
    # This leverages the dynamic search by result feature of Optuna via TPE sampler,
    # where past trial outcomes influence future parameter suggestions.
    score = cross_val_score(classifier, X, y, cv=5, scoring="accuracy").mean()
    
    return score


In [4]:
# ---------------------------------------------------------------------------------
# Step 4: Create an Optuna study and optimize the objective function
# ---------------------------------------------------------------------------------

# Create a study object to maximize the accuracy score
study = optuna.create_study(direction="maximize")

# Optimize the study with a specified number of trials
study.optimize(objective, n_trials=30)

[I 2025-03-05 21:54:31,608] A new study created in memory with name: no-name-f8dbb89c-4674-41ce-b137-ee5658af56b0
[I 2025-03-05 21:54:32,065] Trial 0 finished with value: 0.6510482981071216 and parameters: {'classifier': 'svm', 'svm_C': 0.009581287349012776, 'svm_kernel': 'rbf', 'svm_gamma': 0.00036953209416186727}. Best is trial 0 with value: 0.6510482981071216.
[I 2025-03-05 21:54:34,207] Trial 1 finished with value: 0.7553009082420846 and parameters: {'classifier': 'xgb', 'xgb_n_estimators': 219, 'xgb_max_depth': 10, 'xgb_learning_rate': 0.040584318264051726, 'xgb_subsample': 0.5729003990149821}. Best is trial 1 with value: 0.7553009082420846.
[I 2025-03-05 21:54:34,831] Trial 2 finished with value: 0.6510482981071216 and parameters: {'classifier': 'svm', 'svm_C': 0.09876688467870595, 'svm_kernel': 'rbf', 'svm_gamma': 0.0032568395646150423}. Best is trial 1 with value: 0.7553009082420846.
[I 2025-03-05 21:54:36,343] Trial 3 finished with value: 0.7604872251931075 and parameters: {'c

In [5]:
# ---------------------------------------------------------------------------------
# Step 5: Display the best parameters and corresponding accuracy
# ---------------------------------------------------------------------------------

print(f"Best parameters: {study.best_params}")
print(f"Best accuracy: {study.best_value}")
print(study.trials_dataframe()['params_classifier'].value_counts())
print(study.trials_dataframe().groupby('params_classifier')['value'].mean())


Best parameters: {'classifier': 'xgb', 'xgb_n_estimators': 202, 'xgb_max_depth': 6, 'xgb_learning_rate': 0.012900373670698496, 'xgb_subsample': 0.6301088650031119}
Best accuracy: 0.7722179781003311
params_classifier
xgb    23
svm     7
Name: count, dtype: int64
params_classifier
svm    0.673940
xgb    0.750349
Name: value, dtype: float64


In [6]:
# !pip install plotly

import plotly
print("Plotly version:", plotly.__version__)

Plotly version: 6.0.0


In [7]:
# ---------------------------------------------------------------------------------
# Step 6 (Optional): Visualize the optimization process
# ---------------------------------------------------------------------------------

import optuna.visualization as vis

# Plot the optimization history (accuracy over trials)
vis.plot_optimization_history(study).show()

In [ ]:
from optuna.visualization import plot_contour

# Create a contour plot showing only the 'xgb_n_estimators' parameter.
plot_contour(study, params=["classifier"]).show()

[W 2025-03-05 22:02:53,640] The length of params must be greater than 1.


In [8]:
from optuna.visualization import plot_contour

plot_contour(study).show()

[W 2025-03-05 21:55:07,760] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,760] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,768] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,768] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,768] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,768] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,768] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,776] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,776] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,776] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,776] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,784] Param svm_kernel unique value length is less than 2.
[W 2025-03-05 21:55:07,784] 

In [9]:
# Plot the parallel coordinate visualization to understand parameter interactions
vis.plot_parallel_coordinate(study).show()

[W 2025-03-05 21:55:09,509] Your study has only completed trials with missing parameters.
